### NICO

Implementation based on the matlab code by M. Rabbat

In [1]:
import numpy as np
from random import random, shuffle
from scipy.special import comb
from scipy.sparse import csr_matrix
import itertools

Initialize parameters

In [13]:
n = 4 #number of nodes in the network
T = 100 #number of paths
Nm = 5 #number of nodes per path
np.random.seed(2)

In [14]:
A = np.random.rand(n, n)
A = A / A.sum(axis = 1, keepdims=True)

In [15]:
pi = np.random.rand(n, 1)
pi = pi / pi.sum(axis=0, keepdims=True)

Generate some paths according to this Markov model

In [16]:
X = np.zeros((T, Nm))

In [17]:
# Generate random numbers for testing purposes. Change to random() when it is ready
R_out = np.random.rand(T,1)
R_in = np.random.rand(T, Nm)

In [18]:
cumprobs = pi.cumsum(axis = 0)

In [19]:
iterator = 0
for walk in X:
#     Sample the starting node from Pi
#     larger = (cumprobs >= random()).nonzero()
    larger = (cumprobs >= R_out[iterator][0]).nonzero()
    walk[0] = larger[0][0]
#     Sample remaining nodes in the path by taking a random walk
    for i in range(1, Nm):
        cumprobs_in = A[int(walk[i - 1]),:].cumsum(axis=0)
#       larger = (cumprobs >= random()).nonzero()
        larger = (cumprobs_in >= R_in[iterator][i]).nonzero()
        walk[i] = larger[0][0]
    iterator += 1

Shuffle observations

In [9]:
Y = X.copy()
for walk in Y:
    shuffle(walk)

In [10]:
numTrials = 50

TESTBED

In [ ]:
k = np.array([2,2,1,2])
a = np.array(list(itertools.combinations(k, 2)))
a
# comb(4,2)

In [ ]:
ii = []
jj = []

for walk in X:
    V = np.array(list(itertools.combinations(walk, 2)))
    ii.append(list(V[:, 0]))
    jj.append(list(V[:, 1]))
    
ii = [item for sublist in ii for item in sublist]
jj = [item for sublist in jj for item in sublist]
ss = np.ones(len(ii))

In [ ]:
m = csr_matrix((ss, (ii, jj)), shape = (n,n))

In [ ]:
m.toarray()

In [ ]:
m.transpose().toarray()

In [ ]:
m = m+m.transpose()/2

In [ ]:
m.toarray()

In [ ]:
copy = m.copy()
copy.data.fill(1)

In [ ]:
copy.toarray()

In [ ]:
csr_matrix((np.random.rand(m.nnz),m.nonzero()),shape=m.shape).toarray()

In [ ]:
m = copy + 0.4 * csr_matrix((np.random.rand(m.nnz),m.nonzero()),shape=m.shape)

In [ ]:
m.toarray()

In [ ]:
# m = m / m.sum(axis = 1)

In [ ]:
row_sums = np.array(m.sum(axis=1))[:,0]
row_indices, col_indices = m.nonzero()
m.data /= row_sums[row_indices]

In [ ]:
m.toarray()

In [ ]:
tol = 0.01
kMax = 100
plcutoff = 10
#RAlpha [100x2]
#gamma [4]
#Gamma [4x4]

Utils

In [ ]:
def normalize_csr_rows(csr_mat):
    row_sums = np.array(csr_mat.sum(axis=1))[:,0]
    row_indices, col_indices = csr_mat.nonzero()
    csr_mat.data /= row_sums[row_indices]

NICO implementation

In [ ]:
def nico(X, n):
#     T = np.shape(X)[0]
    
    #number of nodes in each path
#     size = lambda array: len(array)
#     Nm = np.apply_along_axis(size, 1, X)
    
    #Init pi_hat
    #Assume all states appear at least once in the data
    pi_hat = 1 + 0.3 * np.random.rand(n, 1)
    pi_hat = pi_hat / pi_hat.sum(axis = 0, keepdims = True)
    
    # Construct A_hat as a sparse matrix
    # First determine an upperbound on the number of non-zero entries
    ii = []
    jj = []

    for walk in X:
        V = np.array(list(itertools.combinations(walk, 2)))
        ii.append(list(V[:, 0]))
        jj.append(list(V[:, 1]))

    ii = [item for sublist in ii for item in sublist]
    jj = [item for sublist in jj for item in sublist]
    ss = np.ones(len(ii))
    
    A_hat = csr_matrix((ss, (ii, jj)), shape = (n,n))
    A_hat = A_hat + A_hat.transpose() / 2
    A_hat_copy = A_hat.copy()
    A_hat_copy.data.fill(1)
    
    A_hat = A_hat_copy + 0.4 * csr_matrix((np.random.rand(A_hat.nnz),A_hat.nonzero()),shape=A_hat.shape)
    
    #Normalize A_hat
    normalize_csr_rows(A_hat)
#     row_sums = np.array(A_hat.sum(axis=1))[:,0]
#     row_indices, col_indices = A_hat.nonzero()
#     A_hat.data /= row_sums[row_indices]
    
    return A_hat, pi_hat

In [ ]:
nico(X, n)[0].toarray()

In [ ]:
nico(X, n)[1]